##Oxford_flowers102:
Classification Problem to identify flowers out of 102 flowers classes.

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e5279fa8-7912-5fa3-64e3-51e3e723cb51)


In [2]:
!wget https://raw.githubusercontent.com/TahaCycle/ML/main/helper_function.py

--2022-07-01 13:08:36--  https://raw.githubusercontent.com/TahaCycle/ML/main/helper_function.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_function.py’

helper_function.py  100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-01 13:08:36 (92.7 MB/s) - ‘helper_function.py’ saved [10246/10246]



#Importing Libraries


In [3]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import helper_function
import tensorflow_datasets as tfds

##Importing Data Set

In [4]:
ds_list = tfds.list_builders()
(train_data, validation_data, test_data), ds_info = tfds.load(name="oxford_flowers102", # target dataset to get from TFDS
                                                              split=["train", "validation", "test"], # what splits of data should we get? note: not all datasets have train, valid, test
                                                              shuffle_files=True, # shuffle files on download?
                                                              as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                                              with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete4ZJ96W/oxford_flowers102-train.tfrecord


...

  0%|          | 0/6149 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incomplete4ZJ96W/oxford_flowers102-validation.tfrecord


  0%|          | 0/1020 [00:00<?, ? examples/s]

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [5]:
ds_info.features
# Get class names
class_names = ds_info.features["label"].names


In [6]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32' and reshapes image to
  [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

In [7]:
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to validation data
validation_data = validation_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn validation data into batches (don't need to shuffle)
validation_data = validation_data.batch(32).prefetch(tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [8]:
train_data, validation_data, test_data

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [9]:
from helper_function import create_tensorboard_callback

# Create ModelCheckpoint callback to save model's progress
checkpoint_path = "model_checkpoints/cp.ckpt" # saving weights requires ".ckpt" extension
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      montior="val_acc", # save the model weights with best validation accuracy
                                                      save_best_only=True, # only save the best weights
                                                      save_weights_only=True, # only save model weights (not whole model)
                                                      verbose=0) # don't print out whether or not model is being saved 

In [10]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [11]:
mixed_precision.global_policy() # should output "mixed_float16"

<Policy "mixed_float16">

In [12]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False # freeze base model layers

# Create Functional model 
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
# x = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False) # set base_model to inference mode only
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x) # want one output neuron per class 
# Separate activation of output layer so we can output float32 activations
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x) 
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

16719872/16705208 [==============================] - 0s 0us/step


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense (Dense)               (None, 102)               130662    
                                                                 
 softmax_float32 (Activation  (None, 102)              0         
 )                                                               
                                                                 
Total params: 4,180,233
Trainable params: 130,662
Non-trainab

In [15]:
history_101_food_classes_feature_extract = model.fit(train_data, 
                                                     epochs=25,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs", 
                                                                                            "efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

Saving TensorBoard log files to: training_logs/efficientnetb0_101_classes_all_data_feature_extract/20220701-131022
Epoch 1/25
32/32 [==============================] - 6s 116ms/step - loss: 1.2204 - accuracy: 0.9343 - val_loss: 1.6737 - val_accuracy: 0.7455
Epoch 2/25
32/32 [==============================] - 9s 222ms/step - loss: 0.8803 - accuracy: 0.9608 - val_loss: 1.4651 - val_accuracy: 0.7645
Epoch 3/25
32/32 [==============================] - 5s 113ms/step - loss: 0.6630 - accuracy: 0.9735 - val_loss: 1.3096 - val_accuracy: 0.7824
Epoch 4/25
32/32 [==============================] - 5s 115ms/step - loss: 0.5177 - accuracy: 0.9824 - val_loss: 1.2144 - val_accuracy: 0.7879
Epoch 5/25
32/32 [==============================] - 5s 116ms/step - loss: 0.4145 - accuracy: 0.9902 - val_loss: 1.1258 - val_accuracy: 0.8013
Epoch 6/25
32/32 [==============================] - 6s 145ms/step - loss: 0.3411 - accuracy: 0.9912 - val_loss: 1.0687 - val_accuracy: 0.8069
Epoch 7/25
32/32 [===============

In [16]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model


193/193 [==============================] - 12s 64ms/step - loss: 0.6764 - accuracy: 0.8507


[0.6764336824417114, 0.8507074117660522]

In [17]:
# Clone the model we created (this resets all weights)
cloned_model = tf.keras.models.clone_model(model)
cloned_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 pooling_layer (GlobalAverag  (None, 1280)             0         
 ePooling2D)                                                     
                                                                 
 dense (Dense)               (None, 102)               130662    
                                                                 
 softmax_float32 (Activation  (None, 102)              0         
 )                                                               
                                                                 
Total params: 4,180,233
Trainable params: 130,662
Non-trainab

In [18]:
# Load checkpointed weights into cloned_model
cloned_model.load_weights(checkpoint_path)

In [20]:
## Saving model to Google Drive (optional)

# Create save path to drive 
save_dir = "/content/drive/MyDrive/Colab Notebooks"
# os.makedirs(save_dir) # Make directory if it doesn't exist

# Save model
model.save(save_dir)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/assets


In [21]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [22]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [24]:
model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

In [25]:
# Start to fine-tune (all layers)
history_101_food_classes_all_data_fine_tune = model.fit(train_data,
                                                        epochs=100, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # track the model training logs
                                                                   model_checkpoint, # save only the best model during training
                                                                   early_stopping, # stop model after X epochs of no improvements
                                                                   reduce_lr]) # reduce the learning rate after X epochs of no improvements

Saving TensorBoard log files to: training_logs/efficientb0_101_classes_all_data_fine_tuning/20220701-132355
Epoch 1/100
31/32 [============================>.] - ETA: 0s - loss: 0.0428 - accuracy: 1.0000INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


32/32 [==============================] - 47s 1s/step - loss: 0.0428 - accuracy: 1.0000 - val_loss: 0.7497 - val_accuracy: 0.8270 - lr: 1.0000e-04
Epoch 2/100
32/32 [==============================] - ETA: 0s - loss: 0.0406 - accuracy: 1.0000INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


32/32 [==============================] - 42s 1s/step - loss: 0.0406 - accuracy: 1.0000 - val_loss: 0.7457 - val_accuracy: 0.8259 - lr: 1.0000e-04
Epoch 3/100
31/32 [============================>.] - ETA: 0s - loss: 0.0390 - accuracy: 1.0000INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


INFO:tensorflow:Assets written to: fine_tune_checkpoints/assets


32/32 [==============================] - 41s 1s/step - loss: 0.0391 - accuracy: 1.0000 - val_loss: 0.7423 - val_accuracy: 0.8259 - lr: 1.0000e-04
Epoch 4/100
32/32 [==============================] - ETA: 0s - loss: 0.0376 - accuracy: 1.0000

KeyboardInterrupt: ignored